In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer, gen_rbf_X, create_clus_split
from spe.tree import Tree
from spe.forest import BlurredForest

In [2]:
err_cmp = ErrorComparer()

In [3]:
niter = 100

n=20**2
p=30
s=30

m = 20

rho = 0.5
c = np.zeros(n)
c[:m] = [rho**i for i in np.arange(m)]
Sigma_t = toeplitz(c)

# b = n / m
# c = [rho**i for i in np.arange(m)] 
# Sigma_t = block_diag(*[toeplitz(c) for _ in np.arange(b)])

Chol_t = np.linalg.cholesky(Sigma_t)
# Sigma_t = np.eye(n)
# Chol_t = np.eye(n)

k=5
max_depth=4
n_estimators=40

In [4]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()

# c_x = np.random.uniform(0,10,size=n)
# c_y = np.random.uniform(0,10,size=n)

# rn = int(np.sqrt(n))
# ctr_x = np.random.uniform(0,10,size=rn)
# ctr_y = np.random.uniform(0,10,size=rn)
# c_x = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_x])
# c_y = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_y])

# X = gen_rbf_X(c_x, c_y, p)
X = np.random.randn(n,p)
beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [5]:
# xv, yv = np.meshgrid(np.arange(nx), np.arange(ny))
# pts = np.stack([xv.ravel(), yv.ravel()]).T
# n = nx*ny
# rn = int(np.sqrt(n))
# ctr = np.random.choice(pts.shape[0], size=rn, replace=True)
# ctr = pts[ctr]
# tr_idx = np.vstack([[pt + np.array((1.25*np.random.randn(2)).astype(int)) for _ in np.arange(rn)] for pt in ctr])
# tr_idx = np.maximum(0, tr_idx)
# tr_idx[:,0] = cx = np.minimum(nx-1, tr_idx[:,0])
# tr_idx[:,1] = cy = np.minimum(ny-1, tr_idx[:,1])
# tr_idx = np.unique(np.ravel_multi_index(tr_idx.T, (nx,ny)))
# tr_idx.shape

In [6]:
(test_err,
 kfcv_err,
 spcv_err,
 rela_err) = err_cmp.compareForestTrTs(n=n,
                                     p=p,
                                     X=X,
                                     beta=beta,
                                     coord=np.stack([c_x, c_y]).T,
                                     Chol_t=Chol_t,
                                     niter=niter,
                                     max_depth=max_depth,
                                     n_estimators=n_estimators,
                                     k=k)
    

0
0.255


AttributeError: 'BlurredForest' object has no attribute 'eps_'

In [ ]:
%debug

In [ ]:
# groups = KMeans(n_clusters=10).fit(tr_idx).labels_

In [ ]:
# px.scatter(pd.DataFrame({'x': tr_idx[:,0], 'y': tr_idx[:,1], 'g': groups}), x='x', y='y', color='g')

In [ ]:
risk = test_err.mean()
risk_kfcv = kfcv_err.mean()
risk_spcv = spcv_err.mean()
risk_rela = rela_err.mean()
risk, risk_kfcv, risk_spcv, risk_rela

In [ ]:
df = pd.DataFrame({'GenCp': (rela_err.T - test_err.T) / test_err.T,
                   'KFCV': (kfcv_err.T - test_err.T) / test_err.T,
                   'SPCV': (spcv_err.T - test_err.T) / test_err.T})

In [ ]:
fig_rela = px.box((df),# - test_err.mean(1)) / test_err.mean(1), 
             labels={
                     "variable": "Method",
                     "value": "Relative MSE"
                     },
             title="lin correction",
             points=False)
fig_rela.update_traces(boxmean=True)

fig_rela.add_hline(y=0., line_color='red')